<a href="https://colab.research.google.com/github/haneefmohammad/Mask_Detection/blob/main/masktraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths


from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset= r'/content/drive/My Drive/facemask-detector'


In [ ]:
imagePaths=list(paths.list_images(dataset))
imagePaths

['/content/drive/My Drive/facemask-detector/maskexample.jpg',
 '/content/drive/My Drive/facemask-detector/example_01.png',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/417.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/augmented_image_292.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/86.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/442.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/augmented_image_25.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/augmented_image_73.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/151.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/augmented_image_245.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/185.jpg',
 '/content/drive/My Drive/facemask-detector/mask_dataset/without_mask/353.jpg',
 '/content/drive

In [ ]:
data = [ ]
labels = [ ]

In [ ]:
for i in imagePaths:
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)
data=np.array(data,dtype='float32')
labels=np.array(labels)

In [ ]:
data.shape

(1381, 224, 224, 3)

In [ ]:
labels

array(['facemask-detector', 'facemask-detector', 'without_mask', ...,
       'with_mask', 'with_mask', 'with_mask'], dtype='<U17')

In [ ]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

NameError: ignored

In [ ]:
train_X,test_X,train_Y,test_Y=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=10)

ValueError: ignored

In [ ]:
train_X.shape

(1103, 224, 224, 3)

In [ ]:
train_Y.shape

(1103, 2)

In [ ]:
test_X.shape

(276, 224, 224, 3)

In [ ]:
test_Y.shape

(276, 2)

In [ ]:
aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

In [ ]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

In [ ]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

model=Model(inputs=baseModel.input,outputs=headModel)

In [ ]:
for layer in baseModel.layers:
  layer.trainable=False

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [ ]:
learning_rate=0.001
Epochs=20
BS=12

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

H=model.fit(
    aug.flow(train_X,train_Y,batch_size=BS),
    steps_per_epoch=len(train_X)//BS,
    validation_data=(test_X,test_Y),
    validation_steps=len(test_X)//BS,
    epochs=Epochs
)

model.save(r'/content/drive/My Drive/facemask-detector/mobilenet_v2.model')

Epoch 1/20
91/91 [==============================] - 60s 655ms/step - loss: 0.0079 - accuracy: 0.9963 - val_loss: 0.0696 - val_accuracy: 0.9891
Epoch 2/20
91/91 [==============================] - 57s 630ms/step - loss: 0.0397 - accuracy: 0.9936 - val_loss: 0.0663 - val_accuracy: 0.9928
Epoch 3/20
91/91 [==============================] - 58s 634ms/step - loss: 0.0296 - accuracy: 0.9954 - val_loss: 0.0723 - val_accuracy: 0.9928
Epoch 4/20
91/91 [==============================] - 57s 631ms/step - loss: 0.0387 - accuracy: 0.9927 - val_loss: 0.0761 - val_accuracy: 0.9928
Epoch 5/20
91/91 [==============================] - 58s 637ms/step - loss: 0.0243 - accuracy: 0.9936 - val_loss: 0.0741 - val_accuracy: 0.9855
Epoch 6/20
91/91 [==============================] - 58s 642ms/step - loss: 0.0222 - accuracy: 0.9936 - val_loss: 0.0774 - val_accuracy: 0.9928
Epoch 7/20
91/91 [==============================] - 62s 679ms/step - loss: 0.0163 - accuracy: 0.9945 - val_loss: 0.0734 - val_accuracy: 0.9891

FailedPreconditionError: ignored

In [ ]:
predict=model.predict(test_X,batch_size=BS)
predict=np.argmax(predict,axis=1)
print(classification_report(test_Y.argmax(axis=1),predict,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       139
without_mask       0.99      0.99      0.99       137

    accuracy                           0.99       276
   macro avg       0.99      0.99      0.99       276
weighted avg       0.99      0.99      0.99       276

